In [1]:
import os
import sys
import csv

import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# list shp files recursively
os.chdir('..')
abs_path = os.getcwd()

HIFLD_path = os.path.join(abs_path, 'data/HIFLD')
shp_files = [os.path.join(root, name) \
             for root, dirs, files in os.walk(HIFLD_path) \
             for name in files \
             if name.endswith(('.shp'))]

In [3]:
# number of observations per geoshape file 
for file in shp_files:
    with open(file, 'rb') as f:
        row_count = len(f.readlines()) - 1
        
        basename = os.path.basename(file).split('/')[0]
        fname = os.path.basename(basename).split('.')[0]
        
        print('There are {} observations in {}.'.format(row_count, fname))

There are 267622 observations in AllPlacesOfWorship.
There are 7167 observations in CollegesUniversities.
There are 93348 observations in FDIC_Insured_Banks.
There are 54785 observations in Fire_Stations.
There are 8655 observations in Hospitals.
There are 5056 observations in Prison_Boundaries.
There are 112783 observations in PublicSchools.
There are 5298 observations in UrgentCareFacs.


# Explore Geoshape Files
* Columns
* Data types
* CRS

In [4]:
def read_shp(file, rows=100):
    """
    Read geoshapes file

    Parameters
    ----------
        rows (int): number of rows per file to read

    Returns
    -------
        df (GeoDataFrame)
    """
    df = gpd.read_file(file, rows=rows)

    return df

In [5]:
# read all geoshape files into dictionary
dict_geo = {}
for file in shp_files:
    basename = os.path.basename(file).split('/')[0]
    fname = os.path.basename(basename).split('.')[0]

    # print(file)
    dict_geo[fname] = read_shp(file, rows=5000)

## Columns

In [6]:
dict_geo['FDIC_Insured_Banks'].columns

Index(['OBJECTID', 'ADDRESBR', 'BRNUM', 'BRSERTYP', 'CBSABR', 'CBSANAMB',
       'CITYBR', 'CNTRYNAB', 'CNTYNAMB', 'DEPSUMBR', 'GEOCODE_CE', 'NAMEBR',
       'STALPBR', 'STCNTYBR', 'STNAMEBR', 'UNINUMBR', 'ZIPBR', 'CERT',
       'ADDRESS', 'ASSET', 'BKCLASS', 'CITY', 'CNTRYNA', 'DENOVO', 'DEPDOM',
       'NAMEFULL', 'NAMEHCR', 'REGAGNT', 'REPDTE', 'RSSDID', 'STALP', 'STCNTY',
       'STNAME', 'ZIP', 'BKMO', 'LOC_NAME', 'STATUS', 'SCORE', 'x', 'y',
       'GeocodeSou', 'STD_ADDR_B', 'STD_ADDR', 'ZIP4BR', 'geometry'],
      dtype='object')

In [7]:
dict_geo['Prison_Boundaries'].columns

Index(['FID', 'FACILITYID', 'NAME', 'ADDRESS', 'CITY', 'STATE', 'ZIP', 'ZIP4',
       'TELEPHONE', 'TYPE', 'STATUS', 'POPULATION', 'COUNTY', 'COUNTYFIPS',
       'COUNTRY', 'NAICS_CODE', 'NAICS_DESC', 'SOURCE', 'SOURCEDATE',
       'VAL_METHOD', 'VAL_DATE', 'WEBSITE', 'SECURELVL', 'CAPACITY',
       'SHAPE_Leng', 'GlobalID', 'CreationDa', 'Creator', 'EditDate', 'Editor',
       'SHAPE_Le_1', 'SHAPE_Area', 'geometry'],
      dtype='object')

In [8]:
dict_geo['Fire_Stations'].columns

Index(['OBJECTID', 'PERMANENT_', 'SOURCE_FEA', 'SOURCE_DAT', 'SOURCE_D_1',
       'SOURCE_ORI', 'DATA_SECUR', 'DISTRIBUTI', 'LOADDATE', 'FTYPE', 'FCODE',
       'NAME', 'ISLANDMARK', 'POINTLOCAT', 'ADMINTYPE', 'ADDRESSBUI',
       'ADDRESS', 'CITY', 'STATE', 'ZIPCODE', 'GNIS_ID', 'FOOT_ID',
       'COMPLEX_ID', 'GLOBALID', 'geometry'],
      dtype='object')

## CRS

In [9]:
dict_geo['FDIC_Insured_Banks'].crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [10]:
dict_geo['Prison_Boundaries'].crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

# Full Address String
* Output full address string (Street, City, State ZIP) for each GeoDataFrame

In [11]:
def full_address(df):
    """
    Concatenate each address component to a full address string
         (Street, City, State ZIP)
         
    Parameters
    ----------
        df (GeoDataFrame)

    Returns
    -------
        df (GeoDataFrame): with 'full_address' column
    """
    if 'ZIP' in df.columns:
        zip_col = 'ZIP'
    else:
        zip_col = 'ZIPCODE'

    if 'STATE' in df.columns:
        state_col = 'STATE'
    else:
        state_col = 'STNAME'

    if 'ADDRESS' in df.columns:
        addr_col = 'ADDRESS'
    elif 'ADDRESS' not in df.columns:
        addr_col = 'STREET'
    elif ('STREET' not in df.columns) and ('ADDRESS' not in df.columns):
        addr_col = 'STD_ADDR_B'

    # add 0 to zip code if only 4 digits
    df['zip'] = df[zip_col].astype('str')
    df['zip'] = df['zip'].apply(lambda x: '0' + x if len(x) <5 else x)
    
    df[addr_col] = df[addr_col].astype('str')
    df['CITY'] = df['CITY'].astype('str')
    df[state_col] = df[state_col].astype('str')
    
    df['Full_Address'] = df[[addr_col, 'CITY', state_col]].fillna('NaN').agg(', '.join, axis=1) + ' ' + df['zip']
    df['Full_Address'] = df['Full_Address'].astype('str')
             
    return df

In [12]:
%%time
dict_address = {}
for fname in dict_geo:
    print(fname)
    dict_address[fname] = full_address(dict_geo[fname])

AllPlacesOfWorship
CollegesUniversities
FDIC_Insured_Banks
Fire_Stations
Hospitals
Prison_Boundaries
PublicSchools
UrgentCareFacs
CPU times: total: 125 ms
Wall time: 320 ms


# Calculate Latitutde / Longitude
1. Convert each GeoDataFrame to 'EPSG:4326'
2. Calculate centroids from geometry in each GeoDataFrame
3. Extract latitude and longitude from each centroid

In [13]:
def convert_EPSG4326(dict):
    """
    Convert each GeoDataFrame to 'EPSG:4326'
         
    Parameters
    ----------
        dict (dictionary): of GeoDataFrames

    Returns
    -------
        dict (dictionary): GeoDataFrames of 'EPSG:4326' CRS
    """
    for fname in dict:
        dict[fname] = dict[fname].to_crs("EPSG:4326")

    return dict

In [14]:
def get_centroid(dict):
    """
    Convert each GeoDataFrame to 'EPSG:4326'
         
    Parameters
    ----------
        dict (dictionary): GeoDataFrames of 'EPSG:4326' CRS

    Returns
    -------
        dict (dictionary): GeoDataFrames with extracted centroids from geoshapes
    """
    dict_centroids = {}
    for fname in dict:
        if 'x' in dict[fname].columns:
            dict[fname] = dict[fname].rename(columns={'x': 'source_lon', 'y': 'source_lat'})
            dict[fname]['Place_type'] = os.path.basename(fname)
            dict[fname]['source_centroid'] = dict[fname]['geometry']
        else:
            dict[fname]['source_centroid'] = dict[fname]['geometry'].centroid
            dict[fname]['source_lon'] = dict[fname]['geometry'].centroid.x
            dict[fname]['source_lat'] = dict[fname]['geometry'].centroid.y
            dict[fname]['Place_type'] = os.path.basename(fname)

        dict_centroids[fname] = keep_columns(dict[fname])
        
    return dict_centroids

In [15]:
def keep_columns(df):
    """
    Convert each GeoDataFrame to 'EPSG:4326'
         
    Parameters
    ----------
        df (GeoDataFrame): GeoDataFrames of 'EPSG:4326' CRS with extracted centroids

    Returns
    -------
        new_df (GeoDataFrame): GeoDataFrames with only source centroid and full address columns
    """
    cols = ['Full_Address', 'Place_type', 'source_centroid', 'source_lon', 'source_lat']
    new_df = df[cols]

    return new_df

In [16]:
# if you get an error on the first pass, try re-running the cell
dict_EPSG4326 = convert_EPSG4326(dict_address)
dict_centroid = get_centroid(dict_EPSG4326)

C:\Users\bchan\AppData\Local\Temp\ipykernel_10708\4286707561.py:20: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  dict[fname]['source_centroid'] = dict[fname]['geometry'].centroid
C:\Users\bchan\AppData\Local\Temp\ipykernel_10708\4286707561.py:21: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  dict[fname]['source_lon'] = dict[fname]['geometry'].centroid.x
C:\Users\bchan\AppData\Local\Temp\ipykernel_10708\4286707561.py:22: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  dict[fname]['source_lat'] = dict[fname]['geometry'].centroid.y
C:\Users\bchan\AppData\Local\Temp\ipykernel

In [17]:
# check centroids
dict_centroid['FDIC_Insured_Banks']

,Full_Address,Place_type,source_centroid,source_lon,source_lat
0,"100 North Tryon St, Charlotte, North Carolina ...",FDIC_Insured_Banks,POINT (-72.87865 41.60144),-72.878648,41.601435
1,"100 North Tryon St, Charlotte, North Carolina ...",FDIC_Insured_Banks,POINT (-70.25667 43.65743),-70.256668,43.657432
2,"2 Elm Street, Camden, Maine 04843",FDIC_Insured_Banks,POINT (-68.42462 44.54133),-68.424621,44.541326
3,"1 Lincoln St. Fl 1, Boston, Massachusetts 02111",FDIC_Insured_Banks,POINT (-71.05796 42.35282),-71.057959,42.352824
4,"100 North Tryon St, Charlotte, North Carolina ...",FDIC_Insured_Banks,POINT (-71.11891 42.37383),-71.118913,42.373832
...,...,...,...,...,...
4995,"239 Fayetteville Street, Raleigh, North Caroli...",FDIC_Insured_Banks,POINT (-78.19045 38.91756),-78.190446,38.917555
4996,"303 Peachtreet Street, Northeast, Atlanta, Geo...",FDIC_Insured_Banks,POINT (-78.90147 36.69783),-78.901473,36.697827
4997,"5758 Main Street, Auburndale, Wisconsin 54412",FDIC_Insured_Banks,POINT (-90.00738 44.62711),-90.007378,44.627107
4998,"1905 West Stewart Avenue, Wausau, Wisconsin 54401",FDIC_Insured_Banks,POINT (-89.39419 45.63304),-89.394191,45.633037


# Save Geoshape Files

In [18]:
def save_shp(dict, save_dir):
    """
    Save each GeoDataFrame to individual geoshape files
         
    Parameters
    ----------
        dict (dictionary): GeoDataFrames of 'EPSG:4326' CRS with extracted centroids
        save_dir (str): path of desired output directory
    """
    for fname in dict:
        shp_file = dict[fname].set_geometry('source_centroid')
        
        save_path = os.path.join(save_dir, f"{fname}")
        create_dir(save_path)
        
        shp_file.to_file(save_path, driver='ESRI Shapefile')

In [19]:
def create_dir(save_dir):
    """
    Creates directory if it does not exist
         
    Parameters
    ----------
        save_dir (str): path of desired output directory
    """
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

In [20]:
# save files
save_dir = os.path.join(abs_path, 'output/HIFLD/centroids')
create_dir(save_dir)

save_shp(dict_centroid, save_dir)

C:\Users\bchan\AppData\Local\Temp\ipykernel_10708\2389177360.py:16: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  shp_file.to_file(save_path, driver='ESRI Shapefile')
C:\Users\bchan\AppData\Local\Temp\ipykernel_10708\2389177360.py:16: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  shp_file.to_file(save_path, driver='ESRI Shapefile')
C:\Users\bchan\AppData\Local\Temp\ipykernel_10708\2389177360.py:16: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  shp_file.to_file(save_path, driver='ESRI Shapefile')
C:\Users\bchan\AppData\Local\Temp\ipykernel_10708\2389177360.py:16: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  shp_file.to_file(save_path, driver='ESRI Shapefile')
C:\Users\bchan\AppData\Local\Temp\ipykernel_10708\2389177360.py:16: UserWarning: Column names longer than 10

In [21]:
gpd_concat = pd.concat([dict_centroid[fname] for fname in dict_centroid])
gpd_concat

,Full_Address,Place_type,source_centroid,source_lon,source_lat
0,"523 E BROADWAY, SOUTH BOSTON, MA 02127",AllPlacesOfWorship,POINT (-71.04352 42.33547),-71.043522,42.335472
1,"454 ESSEX ST, LAWRENCE, MA 01840",AllPlacesOfWorship,POINT (-71.16494 42.70621),-71.164940,42.706213
2,"569 BROADWAY, NEWARK, NJ 07104",AllPlacesOfWorship,POINT (-74.16282 40.76993),-74.162821,40.769935
3,"3210 SOUTHWESTERN BLVD, ORCHARD PARK, NY 14127",AllPlacesOfWorship,POINT (-78.74782 42.79853),-78.747816,42.798535
4,"431 CAMPGROUND RD, LIVERMORE FLS, ME 04254",AllPlacesOfWorship,POINT (-70.11378 44.42861),-70.113777,44.428610
...,...,...,...,...,...
4805,"1300 PEACHTREE INDUSTRIAL BOULEVARD, SUWANEE, ...",UrgentCareFacs,POINT (-84.09517 34.04173),-84.095174,34.041727
4806,"2660 SATELLITE BOULEVARD NORTHWEST, DULUTH, GA...",UrgentCareFacs,POINT (-84.10132 33.96680),-84.101318,33.966797
4807,"3685 BRASELTON HIGHWAY, DACULA, GA 30019",UrgentCareFacs,POINT (-83.90222 34.06883),-83.902215,34.068832
4808,"1055 DOVE RUN ROAD, LEXINGTON, KY 40502",UrgentCareFacs,POINT (-84.49411 37.99651),-84.494106,37.996508


In [22]:
gpd_concat.to_csv(os.path.join(save_dir, "centroids_40k.csv"), index=False)